## Load NVD3

In [ ]:
from nvd3_stat import Nvd3

In [ ]:
nv = Nvd3()

In [ ]:
nv.reloadNVD3()

In [ ]:
from IPython.display import HTML, display_html, Javascript, display_javascript

display_html(HTML("<style>.container { width:95% !important; }</style>"))

def display(html, style=None):
    if isinstance(html, (tuple, list)):
        html = "".join(["""<div style="display:inline; margin-left:%dpx">%s</div>""" % (el[1], el[0]) for el in html])
    if style is not None:
        html = "<%s>%s</%s>" % (style, html, style)
    display_html(HTML(html))

In [ ]:
# turn javascript debugging on
# nv.nvd3Functions.display_javascript("__nvd3_stat_debug=2")
# turn javascript debugging on
display_javascript(Javascript("__nvd3_stat_debug=0"))


## Some helpers to create data

### Random data

In [ ]:
import numpy as np
import pandas as pd
import random
import time
from datetime import datetime

pd.options.display.width=200

def randomList(count, mean, dist):
    return [ mean + (1 if random.random() > 0.5 else -1) * random.random() * dist for i in range(count)]


def randomNormalList(count, mean, stdev):
    return np.random.normal(mean, stdev, count).tolist()


def randomWalk(count=20, factor=2, offset=10):
    return np.abs(np.cumsum(np.random.uniform(-factor, factor, (count, 1)) ) + offset)


def lastDays(days, epoch=True, ms=True):
    now = datetime.now()
    t = int(time.mktime(now.timetuple())) - 3600 * 24 * days
    if epoch:
        factor = 1000 if ms else 1
        return [int(t + i * 3600 * 24) * factor for i in range(days)]
    else:
        return [time.strftime('%Y-%m-%d', time.localtime(int(t + i*3600*24))) for i in range(days)]

### Stock Downloader

In [ ]:
#
# pip install yahoo-finance
#

from yahoo_finance import Share     

def downloadHistoricalStockData(symbol, begin, end, fillMissing=True):
    data = Share(symbol).get_historical("%04d-%02d-%02d" % begin, "%04d-%02d-%02d" % end)
    df = pd.DataFrame(data)

    for col in "Adj_Close", "Close", "High", "Low", "Open":
        df[col] = df[col].astype(float)
    df["Volume"] = df["Volume"].astype(int)
    df["Date"] = pd.to_datetime(df["Date"], infer_datetime_format=True)
    
    if fillMissing:
        return fillMissingValues(df, symbol, begin, end)
    else: 
        return df

def fillMissingValues(df, symbol, begin, end):
    allDates = pd.DataFrame({"Date": pd.date_range(pd.datetime(*begin), pd.datetime(*end), freq='D')})
    df2 = allDates.merge(df, how="outer")
    df2["Symbol"] = symbol
    df2 = df.fillna(0, axis=1)
    df2["Timestamp"] = (df["Date"].astype("int64") / 1000000).astype('int64')
    return df2.sort_values(by=["Timestamp"])


### Iris data

In [ ]:
# !cd /tmp && wget  https://raw.github.com/pydata/pandas/master/pandas/tests/data/iris.csv

In [ ]:
import pandas as pd
iris = pd.read_csv('/tmp/iris.csv')
print(iris.head())

## 1 BoxPlot Chart

### Compare the three species (IQR box plot)

In [ ]:
bp1 = nv.boxPlotChart()

config={"height": 400, "width":450, "color":nv.c10(),
        "yDomain": [-0.5, 8.5], "maxBoxWidth":False }

def getSpecies(name):
    return iris[iris.Name==name].loc[:,iris.columns != 'Name']

setosa     = bp1.convert(data=getSpecies("Iris-setosa"    ), boxStyle="iqr")
versicolor = bp1.convert(data=getSpecies("Iris-versicolor"), boxStyle="iqr")
virginica  = bp1.convert(data=getSpecies("Iris-virginica" ), boxStyle="iqr")
         
display(html=[("Iris-setosa", 40),("Iris-versicolor", 355),("Iris-virginica", 330)], style="h3")

bp1.plot([
    {"data":setosa,     "config":config},
    {"data":versicolor, "config":config},
    {"data":virginica,  "config":config}
])

### Compare the three species (IQR box plot)

In [ ]:
bp2 = nv.boxPlotChart()

config={"height": 400, "width":450, "color":nv.c10(),
        "yDomain": [-0.5, 6], "maxBoxWidth":False }

display(html=[("IQR", 40),("Min-Max", 415)], style="h3")

iqr =    bp2.convert(data=getSpecies("Iris-setosa"), boxStyle="iqr")
minmax = bp2.convert(data=getSpecies("Iris-setosa"), boxStyle="min-max")

bp2.plot([
    {"data":iqr,     "config":config},
    {"data":minmax,  "config":config}
])         


## 2 Line Chart

In [ ]:
x = np.linspace(0, 4*np.pi, 100)

l_df = pd.DataFrame({"X":x, "Sin":np.sin(x), "Cos":np.cos(x), "ArcTan":np.arctan(x-2*np.pi)/3})
# l_df = pd.DataFrame({"X":lastDays(100, epoch=True, ms=True),"Sin":np.sin(x), "Cos":np.cos(x), "ArcTan":np.arctan(x-2*np.pi)/3})

print(l_df.head())


In [ ]:
l1 = nv.lineChart()

config={"height":500, "width": 1024,
        "focusEnable": False, "color":nv.c20b()[10:13], 
        "yAxis": {"axisLabel":"f(x)", "tickFormat":",.2f"}, 
        "xAxis":{"axisLabel":"x", "tickFormat":",.2f"}}
        
lineAttributes={"area":[True, False, True], "fillOpacity":[0.2, 0, 0.2], "style":["dashed", "dotted", None]}

data = l1.convert(data=l_df[:70], key="X", values=["Sin", "Cos", "ArcTan"],lineAttributes=lineAttributes)        

l1.plot({"data":data, "config":config})

### Dynamically append values

In [ ]:
import time

for i in range(71,100):
    time.sleep(0.1)
    data = l1.convert(data=l_df[i:i+1], key="X", values=["Sin", "Cos", "ArcTan"], lineAttributes=lineAttributes)
    l1.append({"data":data, "config":config})

### Add a focus selector

In [ ]:
l2 = nv.lineChart()

config={"height":500, "width": 1024,
        "focusEnable": True, "color":nv.c10(), 
        "yAxis": {"axisLabel":"f(x)", "tickFormat":",.2f"}, 
        "xAxis":{"axisLabel":"x", "tickFormat":"%d-%m-%Y"}}

lineAttributes={"area":[True, False, True], "fillOpacity":[0.2, 0, 0.2], "style":["dashed", "dotted", None]}
data = l2.convert(l_df, "X", ["Sin", "Cos", "ArcTan"],lineAttributes=lineAttributes) 

l2.plot({"data":data, "config":config})

## 3 Discrete Bar Chart

In [ ]:
db_df = pd.DataFrame(iris.loc[:, iris.columns != "Name"].mean()).reset_index()
db_df.columns = ["Series", "Mean"]

db = nv.discreteBarChart()

config={"height": 350, "width": 500, "color": nv.c20(), "staggerLabels": False}

data = db.convert(db_df, key="Series", value="Mean")

db.plot({"data":data, "config":config})


## 4 Multi Bar Chart

In [ ]:
x = np.linspace(0, 4*np.pi, 10)
mb_df = pd.DataFrame({"X":x, "Sin":np.sin(x), "Cos":np.cos(x), "ArcTan":np.arctan(x-2*np.pi)/3})

mb1 = nv.multiBarChart()

config = {"height":500, "width": 800, 
          "color": nv.c20(),
          "xAxis":{"axisLabel":"x", "tickFormat":",.2f"},
          "yAxis":{"axisLabel":"f(x)", "tickFormat":",.2f"}}

data = mb1.convert(mb_df[:6], "X", ["Sin", "Cos", "ArcTan"])

mb1.plot({"data":data, "config":config})


In [ ]:
for i in range(6,10):
    time.sleep(0.5)
    data = mb1.convert(mb_df[i:i+1], "X", ["Sin", "Cos", "ArcTan"])
    mb1.append({"data":data, "config":config})

## 5 Multi Bar Horizontal Chart

In [ ]:
mbh = nv.multiBarHorizontalChart()

config = {"height":500, "width": 800, "color":nv.c20()[10:], "stacked":False}

data = mbh.convert(mb_df, "X", ["Sin", "Cos", "ArcTan"])

mbh.plot({"data":data,"config":config})


## 6 Line Plus Bar Chart

In [ ]:
hdp = downloadHistoricalStockData('HDP', (2016,3,22), (2017,3,22))
hdp.head(5)


In [ ]:
count = 100
quantity = randomWalk(500, 20, 100)
price = randomWalk(500, 2, 20)
date = lastDays(500)

lpb_df = pd.DataFrame({"Date":date, "Quantity":quantity, "Price":price})

lpb = nv.linePlusBarChart()

config={"color":[nv.c20()[0], nv.c20()[3]], 
        "height":600, "width":1200,
        "xAxis":{"tickFormat":"%d.%m.%y"},
        "x2Axis":{"tickFormat":"%d.%m.%y"}
}

data = lpb.convert(hdp, "Timestamp", lineValue="Close", barValue="Volume")

lpb.plot({"data":data,"config":config})


## 7 Pie Chart

In [ ]:
p = nv.pieChart()

config1={"donut": False,                  "color": nv.c10(),  "width": 300, "height":400}
config2={"donut": True,                   "color": nv.c20(),  "width": 300, "height":400}
config3={"donut": True,  "halfPie": True, "color": nv.c20b(), "width": 300, "height":400}

data = p.convert(db_df, "Series", "Mean")

p.plot([{"data":data,"config":config1},
        {"data":data,"config":config2},
        {"data":data,"config":config3}])

In [ ]:
config = [config1, config2, config3]

for chart in range(3):
    data = p.convert(pd.DataFrame([{"Series":"New", "Mean":3.42}]), "Series", "Mean")
    p.append({"data":data,"config":config[chart]}, chart=chart)

## 8 Stacked Area Chart

In [ ]:
import time
from datetime import datetime

count = 100
groups = 5

series = []
for i in range(groups):
    factor = np.random.randint(5,10)
    offset = np.random.randint(20,100)
    series.append(randomWalk(count, 20, offset).tolist())

sa_df = pd.DataFrame([lastDays(count, epoch=True)] + series).T
sa_df.columns = ["Date"] + ["Series %d" % i  for i in range(groups)]


sa = nv.stackedAreaChart()

config={"color": nv.c20(), "height":500}
data = sa.convert(sa_df, "Date", ["Series %d" % i for i in range(groups)])

sa.plot({"data":data,"config":config})

## 9 Scatter Plus Line Chart

In [ ]:
from sklearn import linear_model

setosa     = iris[iris.Name == "Iris-setosa"]
versicolor = iris[iris.Name == "Iris-versicolor"]
virginica  = iris[iris.Name == "Iris-virginica"]

def linReg(x,y):
    regr = linear_model.LinearRegression()
    regr.fit(x,y)
    return (regr.coef_.item(0), regr.intercept_.item(0))
    
def prepare(df, name):
    sepal = df.loc[:,["SepalLength", "SepalWidth"]]
    sepal.columns = ["X", name]
    sepal["Shape"] = "diamond"
    petal = df.loc[:,["PetalLength", "PetalWidth"]]
    petal.columns = ["X", name]
    petal["Shape"] = "square"
    
    df = pd.concat([sepal, petal])
    x = df["X"].values.reshape(df.shape[0], 1)
    y = df[name].values.reshape(df.shape[0], 1)
    slope, intercept = linReg(x,y)

    df["Size"] = 2
    df = df.groupby(["Shape", "X", name]).sum().reset_index()
    return (df, slope, intercept)

setosaDf,     setosaSlope,     setosaIntercept     = prepare(setosa,     "Setosa")
virginicaDf,  virginicaSlope,  virginicaIntercept  = prepare(virginica,  "Virginica")
versicolorDf, versicolorSlope, versicolorIntercept = prepare(versicolor, "Versicolor")

In [ ]:
spl = nv.scatterPlusLineChart()

config = {"height":700, "xDomain":[0,8.5], "yDomain":[0,4.6],
          "xAxis":{"axisLabel":"Length (sepal=diamond, petal=square)"}, "yAxis":{"axisLabel":"Width (sepal=diamond, petal=square)"}
}

data = spl.convert(data=[{ "data":setosaDf, "keys":"X", "values":"Setosa", 
                           "lines":{"slope":setosaSlope, "intercept":setosaIntercept}, 
                           "pointAttributes":{"shapes":"Shape",  "sizes":"Size"} },
                           
                         { "data":virginicaDf, "keys":"X", "values":"Virginica", 
                           "lines":{"slope":virginicaSlope, "intercept":virginicaIntercept}, 
                           "pointAttributes":{"shapes":"Shape", "sizes":"Size"} },
                           
                         { "data":versicolorDf, "keys":"X", "values":"Versicolor", 
                           "lines":{"slope":versicolorSlope, "intercept":versicolorIntercept}, 
                           "pointAttributes":{"shapes":"Shape", "sizes":"Size"} }
                           ])
                           
spl.plot({"data":data, "config":config})

## 10 Parallel Coordinates Plot

In [ ]:
iris.loc[iris.Name=="Iris-setosa",     "color"] = nv.c10()[0]
iris.loc[iris.Name=="Iris-versicolor", "color"] = nv.c10()[1]
iris.loc[iris.Name=="Iris-virginica",  "color"] = nv.c10()[2]
iris["strokeWidth"] = 0.5
iris.head()


In [ ]:
pc = nv.parallelCoordinatesChart()

config = {"height": 600}

data, dim = pc.convert(iris, 'Name', ['SepalWidth', 'SepalLength', 'PetalWidth', 'PetalLength'],
                       lineAttributes=["color", "strokeWidth"], 
                       dimAttributes= {"format": [",.1f", ",.1f", ",.1f", ",.1f"]})
                       
pc.plot({"data":data, "dim":dim, "config":config})    


## 11 Historical Bar Chart

In [ ]:
hb = nv.historicalBarChart()
hdp["VolumeMio"] = hdp["Volume"] / 10000000

config = {"color":nv.c20()[4:],
          "xAxis":{"axisLabel":"Date (d.m.y)"},
          "yAxis":{"axisLabel":"Volume (Mio)", "tickFormat":",.2f"}}

data = hb.convert(hdp, key="Timestamp", value="VolumeMio")

hb.plot({"data":data, "dim":dim, "config":config})

## 12 Candlestick Chart

In [ ]:
cs = nv.candlestickBarChart()

ohlcAttribs = {"open":"Open" ,"high":"High" ,"low":"Low" ,"volume":"Volume" ,"adjusted":"Adj_Close"}
data = cs.convert(hdp[-90:], "Timestamp", "Close", ohlcAttribs)

config = {"color":nv.c10(), "yDomain":[7,12], "width":1400, "height":800,
          "xAxis":{"tickFormat":"%d/%m/%Y", "axisLabel":"Date (d/m/y)"},
          "yAxis":{"axisLabel":"Close (USD)"}}

cs.plot({"data":data, "config":config})

## 13 OHLC Chart

In [ ]:
ohlc = nv.ohlcBarChart()

ohlcAttribs = {"open":"Open" ,"high":"High" ,"low":"Low" ,"volume":"Volume" ,"adjusted":"Adj_Close"}
data = ohlc.convert(hdp[-90:], "Timestamp", "Close", ohlcAttribs)

config = {"color":nv.c20(), "yDomain":[7,12], "width":1400, "height":800,
          "xAxis":{"tickFormat":"%d/%m/%Y", "axisLabel":"Date (d/m/y)"},
          "yAxis":{"axisLabel":"Open / High / Low / Close  (USD)"}}

ohlc.plot({"data":data, "config":config})

## 14 BulletChart

In [ ]:
def getData(title, actual, previous):
    return {"title":title, "subtitle":"out of 5",
            "ranges":{'Bad':3.5, 'OK':4.25, 'Good':5},
            "measure":{'Current':actual},
            "markers":{'Previous':previous},
            "markerLines":{'Threshold':3.0, 'Target':4.4}}
                
b1 = nv.bulletChart()
data = b1.convert(**getData("Satisfaction", 3.9, 3.8))
b1.plot({"data":data, "config":{"height":60, "width":750}})

b2 = nv.bulletChart()
data = b2.convert(**getData("Quality", 4.3, 3.8))
b2.plot({"data":data, "config":{"height":60, "width":750}})
